# Poglavlje 7: Izrada chat aplikacija
## Brzi početak s OpenAI API-jem

Ova bilježnica je prilagođena iz [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) koja sadrži bilježnice za pristup [Azure OpenAI](notebook-azure-openai.ipynb) uslugama.

Python OpenAI API radi i s Azure OpenAI modelima, uz nekoliko izmjena. Više o razlikama možete saznati ovdje: [Kako prebacivati između OpenAI i Azure OpenAI krajnjih točaka pomoću Pythona](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# Pregled  
"Veliki jezični modeli su funkcije koje preslikavaju tekst u tekst. Kada dobiju ulazni niz teksta, veliki jezični model pokušava predvidjeti koji će tekst slijediti"(1). Ova "brza početna" bilježnica upoznat će korisnike s osnovnim pojmovima vezanim uz LLM-ove, ključnim paketima potrebnim za početak rada s AML-om, laganim uvodom u dizajn upita te nekoliko kratkih primjera različitih slučajeva korištenja.


## Sadržaj

[Pregled](../../../../07-building-chat-applications/python)  
[Kako koristiti OpenAI uslugu](../../../../07-building-chat-applications/python)  
[1. Kreiranje vaše OpenAI usluge](../../../../07-building-chat-applications/python)  
[2. Instalacija](../../../../07-building-chat-applications/python)  
[3. Podaci za prijavu](../../../../07-building-chat-applications/python)  

[Primjeri upotrebe](../../../../07-building-chat-applications/python)  
[1. Sažimanje teksta](../../../../07-building-chat-applications/python)  
[2. Klasifikacija teksta](../../../../07-building-chat-applications/python)  
[3. Generiranje novih imena proizvoda](../../../../07-building-chat-applications/python)  
[4. Fino podešavanje klasifikatora](../../../../07-building-chat-applications/python)  

[Reference](../../../../07-building-chat-applications/python)


### Napravite svoj prvi prompt  
Ova kratka vježba pružit će vam osnovni uvod u slanje promptova OpenAI modelu za jednostavan zadatak "sažimanja".


**Koraci**:  
1. Instalirajte OpenAI biblioteku u svoje python okruženje  
2. Učitajte standardne pomoćne biblioteke i postavite svoje uobičajene OpenAI sigurnosne podatke za OpenAI uslugu koju ste kreirali  
3. Odaberite model za svoj zadatak  
4. Napravite jednostavan prompt za model  
5. Pošaljite svoj zahtjev model API-ju!


### 1. Instalirajte OpenAI


In [ ]:
%pip install openai python-dotenv

### 2. Uvezite pomoćne biblioteke i inicijalizirajte vjerodajnice


In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. Pronalaženje pravog modela  
GPT-3.5-turbo ili GPT-4 modeli mogu razumjeti i generirati prirodni jezik.


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. Dizajn upita  

"Čarolija velikih jezičnih modela je u tome što, trenirajući se na minimiziranju pogreške predviđanja na ogromnim količinama teksta, modeli zapravo usvajaju koncepte korisne za ta predviđanja. Na primjer, nauče koncepte poput"(1):

* kako se pišu riječi
* kako funkcionira gramatika
* kako preformulirati rečenicu
* kako odgovarati na pitanja
* kako voditi razgovor
* kako pisati na raznim jezicima
* kako programirati
* itd.

#### Kako upravljati velikim jezičnim modelom  
"Od svih ulaza u veliki jezični model, najvažniji je tekstualni upit (prompt)(1).

Veliki jezični modeli mogu se potaknuti na generiranje izlaza na nekoliko načina:

Instrukcija: Recite modelu što želite
Dovršavanje: Potaknite model da dovrši početak onoga što želite
Demonstracija: Pokažite modelu što želite, bilo:
Nekoliko primjera u upitu
Stotine ili tisuće primjera u skupu podataka za fino podešavanje"



#### Postoje tri osnovne smjernice za kreiranje upita:

**Pokažite i objasnite.** Jasno dajte do znanja što želite, bilo kroz upute, primjere ili kombinaciju oboje. Ako želite da model rangira popis stavki abecednim redom ili klasificira odlomak prema sentimentu, pokažite mu da je to ono što želite.

**Koristite kvalitetne podatke.** Ako pokušavate izgraditi klasifikator ili navesti model da slijedi određeni obrazac, pobrinite se da ima dovoljno primjera. Obavezno provjerite primjere — model je obično dovoljno pametan da prepozna osnovne pravopisne greške i da vam odgovori, ali može pretpostaviti da je to namjerno i to može utjecati na odgovor.

**Provjerite postavke.** Temperature i top_p postavke određuju koliko je model determinističan u generiranju odgovora. Ako tražite odgovor gdje postoji samo jedan točan odgovor, te vrijednosti treba postaviti niže. Ako želite raznovrsnije odgovore, možda ih želite postaviti više. Najčešća greška kod ovih postavki je pretpostavka da su to kontrole za "domišljatost" ili "kreativnost".


Izvor: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Sažmi tekst  
#### Izazov  
Sažmi tekst tako da na kraj teksta dodaš 'tl;dr:'. Primijeti kako model razumije kako obaviti niz zadataka bez dodatnih uputa. Možeš eksperimentirati s opisnijim promptovima od tl;dr kako bi promijenio ponašanje modela i prilagodio sažetak koji dobiješ(3).  

Nedavna istraživanja pokazala su značajan napredak na brojnim NLP zadacima i testovima zahvaljujući predtreningu na velikom korpusu teksta, nakon čega slijedi fino podešavanje za određeni zadatak. Iako je ova metoda obično neovisna o zadatku po svojoj arhitekturi, i dalje zahtijeva posebne skupove podataka za fino podešavanje s tisućama ili desecima tisuća primjera. S druge strane, ljudi uglavnom mogu obaviti novi jezični zadatak već nakon nekoliko primjera ili jednostavnih uputa – što je nešto s čime se današnji NLP sustavi još uvijek teško nose. Ovdje pokazujemo da povećanje jezičnih modela znatno poboljšava neovisnu izvedbu na zadacima s malo primjera, ponekad čak dosežući konkurentnost s dosadašnjim najboljim pristupima finog podešavanja.  

Tl;dr


# Vježbe za nekoliko slučajeva upotrebe  
1. Sažmi tekst  
2. Klasificiraj tekst  
3. Generiraj nova imena proizvoda


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Klasificiraj tekst  
#### Izazov  
Klasificiraj stavke u kategorije koje se navode prilikom izvođenja. U sljedećem primjeru, u upitu navodimo i kategorije i tekst koji treba klasificirati (*playground_reference).

Upit korisnika: Pozdrav, jedna tipka na mojoj tipkovnici za laptop se nedavno pokvarila i trebat će mi zamjena:

Klasificirana kategorija:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Generiraj nova imena proizvoda
#### Izazov
Stvori imena proizvoda na temelju primjera riječi. U ovom zadatku uključujemo informacije o proizvodu za koji ćemo generirati imena. Također dajemo sličan primjer kako bismo pokazali obrazac koji želimo dobiti. Postavili smo i visoku vrijednost temperature kako bismo povećali nasumičnost i dobili inovativnije odgovore.

Opis proizvoda: Aparat za pripremu milkshakea kod kuće
Ključne riječi: brzo, zdravo, kompaktno.
Imena proizvoda: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Opis proizvoda: Par cipela koje pristaju svakom broju stopala.
Ključne riječi: prilagodljivo, pristaje, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Reference  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Primjeri](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Najbolje prakse za fino podešavanje GPT-3 za klasifikaciju teksta](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Za dodatnu pomoć  
[OpenAI Komercijalni tim](AzureOpenAITeam@microsoft.com)


# Suradnici
* Louis Li



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge prevođenja [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni ljudski prijevod. Ne snosimo odgovornost za bilo kakva nesporazume ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
